In [19]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math

In [2]:
data = pd.read_csv("data.csv")
data

,age,income,student,credit rating,buys computer
0,<=30,high,no,fair,no
1,<=30,high,no,fair,no
2,31..40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes
5,31..40,low,yes,fair,yes
6,31..40,low,yes,fair,no
7,31..40,low,yes,fair,yes
8,<=30,medium,no,fair,no
9,<=30,medium,no,fair,yes


In [3]:
data.dtypes

age              object
income           object
student          object
credit rating    object
buys computer    object
dtype: object

In [4]:
for col in ["student","buys computer"]:
        data[col].replace({"yes":1,"no":0},inplace=True)

In [5]:
data["credit rating"].replace({"excellent":1,"fair":0},inplace=True)

In [6]:
data["income"].replace({"low":1,"medium":2,"high":3},inplace=True)

In [7]:
data["age"].replace({"<=30":1,"31..40":2,">40":3},inplace=True)

In [8]:
data

,age,income,student,credit rating,buys computer
0,1,3,0,0,0
1,1,3,0,0,0
2,2,3,0,0,1
3,3,2,0,0,1
4,3,1,1,0,1
5,2,1,1,0,1
6,2,1,1,0,0
7,2,1,1,0,1
8,1,2,0,0,0
9,1,2,0,0,1


In [9]:
X = data.drop("buys computer", axis=1)

In [10]:
Y = data["buys computer"]

In [16]:
X = np.array(X)
X

array([[1, 3, 0, 0],
       [1, 3, 0, 0],
       [2, 3, 0, 0],
       [3, 2, 0, 0],
       [3, 1, 1, 0],
       [2, 1, 1, 0],
       [2, 1, 1, 0],
       [2, 1, 1, 0],
       [1, 2, 0, 0],
       [1, 2, 0, 0],
       [3, 2, 1, 0],
       [1, 2, 1, 0],
       [1, 2, 1, 1],
       [2, 2, 0, 1],
       [2, 3, 1, 0],
       [3, 2, 0, 1]], dtype=int64)

In [17]:
Y = np.array(Y)
Y

array([0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0], dtype=int64)

In [20]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth #Initializing Max Depth for Stoping Criteria

    def fit(self, X, y, depth=0):
        if depth == self.max_depth or len(set(y)) == 1:
            # If the maximum depth is reached or all labels are the same, create a leaf node
            unique_labels, counts = np.unique(y, return_counts=True)
            return {'class': unique_labels[np.argmax(counts)]}

        best_feature, best_threshold = self.find_best_split(X, y)

        if best_feature is None:
            #if there is no feature to divide node create it to leaf node
            unique_labels, counts = np.unique(y, return_counts=True)
            return {'class': unique_labels[np.argmax(counts)]}

        left_indices = X[:, best_feature] <= best_threshold #Thresholding and adding feature to left node
        right_indices = ~left_indices

        #Recursion
        left_subtree = self.fit(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self.fit(X[right_indices], y[right_indices], depth + 1)

        return {
            'feature_index': best_feature,
            'threshold': best_threshold,
            'left': left_subtree,
            'right': right_subtree
        }


    def find_best_split(self, X, y):
        num_features = X.shape[1]
        best_feature = None
        best_threshold = None
        best_info_gain = 0

        for feature_index in range(num_features):
            thresholds = np.unique(X[:, feature_index])

            for threshold in thresholds:
                left_indices = X[:, feature_index] <= threshold
                right_indices = ~left_indices

                if len(y[left_indices]) == 0 or len(y[right_indices]) == 0:
                    continue

                info_gain = self.calculate_info_gain(y, y[left_indices], y[right_indices])

                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_feature = feature_index
                    best_threshold = threshold

        return best_feature, best_threshold

    def calculate_info_gain(self, parent, left_child, right_child):
        parent_entropy = self.entropy(parent)
        left_child_entropy = (len(left_child) / len(parent)) * self.entropy(left_child)
        right_child_entropy = (len(right_child) / len(parent)) * self.entropy(right_child)

        info_gain = parent_entropy - (left_child_entropy + right_child_entropy)
        return info_gain

    def entropy(self,labels):
        # Count the occurrences of each unique label
        label_counts = {}
        for label in labels:
            if label in label_counts:
              label_counts[label] += 1
            else:
              label_counts[label] = 1

         # Calculate entropy using the formula
         # Entropy(s)= -P(yes)log2 P(yes)- P(no) log2 P(no)
        entropy_value = 0
        total_instances = len(labels)
        for count in label_counts.values():
            probability = count / total_instances
            entropy_value -= probability * math.log2(probability)

            return entropy_value

    def predict_instance(self, instance, tree):
        if 'class' in tree:
            return tree['class']

        if instance[tree['feature_index']] <= tree['threshold']:
            return self.predict_instance(instance, tree['left'])
        else:
            return self.predict_instance(instance, tree['right'])

    def predict(self, X, tree):
        return [self.predict_instance(instance, tree) for instance in X]


In [21]:
tree = DecisionTree(max_depth=6)

In [22]:
model = tree.fit(X, Y)

In [26]:
def visualize_tree(tree, indent=''):
        if 'class' in tree:
            print(F"{indent}{indent}Class: {tree['class']}")
        else:
            print(f"{indent}{indent}Feature {tree['feature_index']} <= {tree['threshold']}")
            print(f"{indent}{indent}True:")
            visualize_tree(tree['left'], indent + '     ')
            print(f"{indent}{indent}False:")
            visualize_tree(tree['right'], indent + '    ')

In [27]:
visualize_tree(model)

Feature 2 <= 0
True:
          Feature 0 <= 1
          True:
                    Feature 1 <= 2
                    True:
                              Class: 0
                    False:
                            Class: 0
          False:
                  Feature 0 <= 2
                  True:
                            Class: 1
                  False:
                          Feature 3 <= 0
                          True:
                                    Class: 1
                          False:
                                  Class: 0
False:
        Feature 1 <= 1
        True:
                  Feature 0 <= 2
                  True:
                            Class: 1
                  False:
                          Class: 1
        False:
                Class: 1
